In [1]:
from Datasets.News.news_data_loader import data_loader
from Datasets.News.news_gen_test import gen_test_data
from utils.cluster import KMEANS
from utils.get_features import SIFT
from utils.find_similar_vectors import KNN
import matplotlib.pyplot as plt
import numpy as np

In [2]:
images, image_names = data_loader()

  0%|          | 0/41762 [00:00<?, ?it/s]

100%|██████████| 41762/41762 [01:59<00:00, 348.93it/s]


In [3]:
sift_detector = SIFT('sift')
train_keypoints, features = sift_detector.get_features(images, True)

features_array = np.concatenate(features, axis=0)

100%|██████████| 41762/41762 [19:13<00:00, 36.21it/s]


In [4]:
num_features, dim = features_array.shape
num_clusters = 500

kmeans = KMEANS(dim, num_clusters=num_clusters, niter=200)
kmeans.train(features_array)

Sampling a subset of 128000 / 40025973 for training
Clustering 128000 points in 128D to 500 clusters, redo 1 times, 200 iterations
  Preprocessing in 5.18 s
  Iteration 199 (11.53 s, search 10.33 s): objective=9.10152e+09 imbalance=1.075 nsplit=0       


In [5]:
from tqdm import tqdm


hists = []

for f in tqdm(features):
    labels = kmeans.search(f)[:, 0]
    counts = np.unique(labels, return_counts=True)
    arr = np.zeros(num_clusters)
    arr[counts[0]] = counts[1]
    arr /= np.linalg.norm(arr)
    hists.append(arr[None, :]) # dim: 1 x num_clusters

hists = np.concatenate(hists, axis=0)    

100%|██████████| 41762/41762 [00:42<00:00, 984.33it/s] 


In [6]:
knn = KNN(num_clusters)
knn.fit(hists)
knn.findKNearest(hists[:3])

array([[    0, 37067, 39414, 40146, 39766,   998, 40258,  4750, 10467,
        40659],
       [    1, 29337, 16279, 39619, 39894, 33187, 18322,  2704, 37769,
          354],
       [    2, 39948, 41430, 41675, 41606, 40258,  5423, 39744, 41758,
        29731]])

# Inferencing with augmented data

In [7]:
test_image_names, test_images = [], []
num_test_images = 1000
for itr, (img, name) in enumerate(gen_test_data()):
    if itr == num_test_images:
        break
    test_images.append(img['image'])
    test_image_names.append(name)
    
test_keypoints, test_features = sift_detector.get_features(test_images, True)

100%|██████████| 1000/1000 [00:22<00:00, 44.04it/s]


In [8]:
K = 5
precision = np.zeros(K)

for itr, (image, name) in enumerate(tqdm(zip(test_images, test_image_names))):    # get the sift features
    f = test_features[itr]    # it is possible that no descriptors are obtained
    if f is None:
        print(f'{name}: no features extracted')
        continue
        
    f = np.array(f)
    
    # histogram
    labels = kmeans.search(f)[:, 0]        
    counts = np.unique(labels, return_counts=True)
    arr = np.zeros(num_clusters)
    arr[counts[0]] = counts[1]
    arr /= np.linalg.norm(arr)
    
    # finding the nearest neighbours
    indices = knn.findKNearest(arr[None, :], K)[0]
    # finding the position of correct image
    # in returned neighbours
    # the original image may not be there in the image
    pos = np.where(indices==itr)[0]
    if pos.size:
        precision[pos[0]:] += 1

1000it [00:14, 71.29it/s]


In [9]:
precision/itr

array([0.75975976, 0.8038038 , 0.81881882, 0.82782783, 0.83483483])

# RANSAC

In [10]:
from utils.ransac import ransac_sift_multiprocessing, ransac_sift


In [11]:
K = 5
precision = np.zeros(K)

for itr, (image, name) in enumerate(tqdm(zip(test_images, test_image_names))):    # get the sift features
    f = test_features[itr]
    kp = test_keypoints[itr]
    # it is possible that no descriptors are obtained
    if f is None:
        print(f'{name}: no features extracted')
        continue
        
    f = np.array(f)
    
    # histogram
    labels = kmeans.search(f)[:, 0]        
    counts = np.unique(labels, return_counts=True)
    arr = np.zeros(num_clusters)
    arr[counts[0]] = counts[1]
    arr /= np.linalg.norm(arr)
    
    # finding the nearest neighbours
    indices = knn.findKNearest(arr[None, :], 5*K)[0]
    # RANSAC
    kps = [train_keypoints[i] for i in indices]
    ds = [features[i] for i in indices]
    
    scores = ransac_sift_multiprocessing(f, ds, kp, kps)
    
    high_score_indices = np.argsort(scores)[::-1][:K]
    indices = indices[high_score_indices]
    
    # finding the position of correct image
    # in returned neighbours
    # the original image may not be there in the image
    pos = np.where(indices==itr)[0]
    if pos.size:
        precision[pos[0]:] += 1

1000it [4:38:21, 16.70s/it]


In [12]:
precision/itr

array([0.77577578, 0.82382382, 0.84284284, 0.84784785, 0.85485485])